You have to work on the files:
*  [Books](https://github.com/gdv/foundationsCS/raw/master/progetti/2021/Books.csv.gz)
*  [Book ratings](https://github.com/gdv/foundationsCS/raw/master/progetti/2021/Book-Ratings.csv.gz)
*  [Users](https://github.com/gdv/foundationsCS/raw/master/progetti/2021/Users.csv.gz)
*  [Goodbooks books](https://github.com/gdv/foundationsCS/raw/master/progetti/2021/goodbooks.csv.gz)
*  [Goodbooks ratings](https://github.com/gdv/foundationsCS/raw/master/progetti/2021/goodbooks-ratings.csv.gz)

### Notes

1.    It is mandatory to use GitHub for developing the project.
1.    The project must be a jupyter notebook.
1.    There is no restriction on the libraries that can be used, nor on the Python version.
1.    To read those files, you need to use the `encoding = 'latin-1'` option.
1.    All questions on the project **must** be asked in a public channel on [Zulip](https://focs.zulipchat.com), otherwise no  answer will be given.

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
import pandas as pd
import numpy as np
import re

### Importo i dataset

Per importare inserisco tra i parametri il compressore da utilizzare, l'encoding, il separatore e dove necessario l'escape character. Inoltre per ogni dataset specifico il tipo di ogni attributo cosí da notare eventuali problemi con qualche riga dei dataframe. Per il dataset goodbooks utilizziamo l'encoding utf-8 che riesce ad interpretare meglio le stringhe

In [3]:
users = pd.read_csv('https://github.com/gdv/foundationsCS/raw/master/progetti/2021/Users.csv.gz', compression = 'gzip', escapechar = "\\", encoding = 'latin-1', sep = ';', dtype = {'Age': 'Int64'})
users.shape

(278858, 3)

In [4]:
books = pd.read_csv('https://github.com/gdv/foundationsCS/raw/master/progetti/2021/Books.csv.gz', compression = 'gzip', escapechar = "\\", encoding = 'latin-1', sep = ';', usecols=['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher'])
books.shape

(271359, 5)

In [5]:
goodbooks = pd.read_csv('https://github.com/gdv/foundationsCS/raw/master/progetti/2021/goodbooks.csv.gz', compression = 'gzip', encoding = 'utf-8', sep = ',', dtype = {'isbn13': 'object', 'original_publication_year': 'int64',})
goodbooks.shape

(99, 23)

In [6]:
book_ratings = pd.read_csv('https://github.com/gdv/foundationsCS/raw/master/progetti/2021/Book-Ratings.csv.gz', compression = 'gzip', escapechar = "\\", encoding = 'latin-1', sep = ';')
book_ratings.shape

(1149780, 3)

In [7]:
goodbooks_ratings = pd.read_csv('https://github.com/gdv/foundationsCS/raw/master/progetti/2021/goodbooks-ratings.csv.gz', compression = 'gzip', encoding = 'latin-1', sep = ',')
goodbooks_ratings.shape

(99, 3)

### Sistemazione dataset
#### Controllo tipologia di attributi dei dataset

In [8]:
users.dtypes

User-ID      int64
Location    object
Age          Int64
dtype: object

In [9]:
books.dtypes

ISBN                   object
Book-Title             object
Book-Author            object
Year-Of-Publication     int64
Publisher              object
dtype: object

In [10]:
goodbooks.dtypes

book_id                        int64
goodreads_book_id              int64
best_book_id                   int64
work_id                        int64
books_count                    int64
isbn                          object
isbn13                        object
authors                       object
original_publication_year      int64
original_title                object
title                         object
language_code                 object
average_rating               float64
ratings_count                  int64
work_ratings_count             int64
work_text_reviews_count        int64
ratings_1                      int64
ratings_2                      int64
ratings_3                      int64
ratings_4                      int64
ratings_5                      int64
image_url                     object
small_image_url               object
dtype: object

In [11]:
book_ratings.dtypes

User-ID         int64
ISBN           object
Book-Rating     int64
dtype: object

In [12]:
goodbooks_ratings.dtypes

user_id    int64
book_id    int64
rating     int64
dtype: object

#### Sistemazione valori anomali

Per quanto riguarda il dataset book_ratings utilizzo la funzione strip per rimuovere eventuali spazi o virgolette agli estremi degli ISBN. Utilizando poi la funzione contains ci si accorge che ci sono un migliaio di righe con spazie e caratteri speciali anche all'interno degli ISBN.

In [13]:
book_ratings['ISBN'] = book_ratings['ISBN'].str.strip(' "')
book_ratings[book_ratings['ISBN'].str.contains("\W+", regex = True)].head()

#book_ratings['ISBN'] = [re.sub('[\W+]', '', s) for s in book_ratings['ISBN']]

,User-ID,ISBN,Book-Rating
535,276929,2.02.032126.2,0
536,276929,2.264.03602.8,0
8918,278491,01420.01740,10
9745,183,100940/86,9
9746,183,10622/86,0


In [14]:
books[books['ISBN'].str.contains("\W+", regex = True)]

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher
111808,0486404242\t,War in Kind: And Other Poems (Dover Thrift Edi...,Stephen Crane,1998,Dover Publications
171206,3518365479<90,"Suhrkamp TaschenbÃ?ÃÂ¼cher, Nr.47, Frost",Thomas Bernhard,1972,Suhrkamp
251423,3442248027 3,Diamond Age. Die Grenzwelt.,Neal Stephenson,2000,Goldmann
251648,0385722206 0,Balzac and the Little Chinese Seamstress : A N...,DAI SIJIE,2002,Anchor


### 1. Normalize the location field of *Users* dataset, splitting into city, region, country.

Il right split permette di dividere le stringhe a partire dal valore Country per tentare di risolvere alcuni problemi con le righe con piú di due virgole nel campo Location.

In [15]:
users[['City', 'Region', 'Country']] = users['Location'].str.rsplit(',', 2, expand=True)
users.head()

,User-ID,Location,Age,City,Region,Country
0,1,"nyc, new york, usa",<NA>,nyc,new york,usa
1,2,"stockton, california, usa",18,stockton,california,usa
2,3,"moscow, yukon territory, russia",<NA>,moscow,yukon territory,russia
3,4,"porto, v.n.gaia, portugal",17,porto,v.n.gaia,portugal
4,5,"farnborough, hants, united kingdom",<NA>,farnborough,hants,united kingdom


Facendo un group by rispetto a Country ci si rende conto della presenza di valori anomali in alcuni campi e inoltre si notano alcuni problemi con l'encoding.

In [16]:
users.groupby('Country').count().head()

,User-ID,Location,Age,City,Region
Country,,,,,
,4588,4588,2000,4588,4588
"""n/a""",2,2,1,2,2
&#20013;&#22269;,1,1,1,1,1
&#32654;&#22269;,1,1,1,1,1
*,1,1,0,1,1


### 2. For each book in the *Books* dataset, compute its average rating.

In [17]:
books_avg = pd.merge(books, book_ratings, on = 'ISBN', how = 'left').groupby(['Book-Title', 'ISBN'], as_index=False).agg(Book_Average_Rating = ('Book-Rating', 'mean'))
books_avg.head()

,Book-Title,ISBN,Book_Average_Rating
0,A Light in the Storm: The Civil War Diary of ...,0590567330,2.25
1,Always Have Popsicles,0964147726,0.00
2,Apple Magic (The Collector's series),0942320093,0.00
3,"Ask Lily (Young Women of Faith: Lily Series, ...",0310232546,8.00
4,Beyond IBM: Leadership Marketing and Finance ...,0962295701,0.00


### 3. For each book in the *GoodBooks* dataset, compute its average rating.

In [18]:
goodbooks[['isbn', 'original_title', 'average_rating']].head()

,isbn,original_title,average_rating
0,439023483,The Hunger Games,4.34
1,439554934,Harry Potter and the Philosopher's Stone,4.44
2,316015849,Twilight,3.57
3,61120081,To Kill a Mockingbird,4.25
4,743273567,The Great Gatsby,3.89


### 4. Merge together all rows sharing the same book title, author and publisher. We will call the resulting datset `merged books`. The books that have not been merged together will not appear in `merged books`.

Creo un dataset merging_books, che sfrutteró anche nel punto successivo, dove con la funzione duplicated con keep = False trovo che si ripetono almeno una volta rispetto ai campi [Book-Title, Book-Author, Publisher]. Successivamente creo il dataset merged_books facendo un group by del dataset precedente rispetto a [Book-Title, Book-Author, Publisher].

In [19]:
merging_books = books[books.duplicated(subset = ['Book-Title', 'Book-Author', 'Publisher'], keep = False)]
merged_books = merging_books.groupby(['Book-Title', 'Book-Author', 'Publisher'], as_index=False).agg(Count=('ISBN', 'count'))
merged_books.head()

,Book-Title,Book-Author,Publisher,Count
0,!%@ (A Nutshell handbook),Donnalyn Frey,O'Reilly,2
1,'A Hell of a Place to Lose a Cow': An American...,Tim Brookes,National Geographic,2
2,"10,000 dreams interpreted: A dictionary of dreams",Gustavus Hindman Miller,Barnes &amp; Nobles Books,2
3,101 Famous Poems,Roy J. Cook,McGraw-Hill/Contemporary Books,3
4,15 Houseplants Even You Can't Kill,Joe Elder,Berkley Pub Group,2


### 5. For each book in `merged books` compute its average rating.

Calcolo l'Average_Rating per ogni edizione dei libri in book_ratings e poi faccio un merge con il dataset creato al punto precedente.

In [20]:
book_ratings_avg = book_ratings.groupby('ISBN', as_index=True).agg(Average_Rating = ('Book-Rating', 'mean'))
merging_book_ratings = pd.merge(merging_books, book_ratings_avg, on = 'ISBN', how = 'left').sort_values(['Book-Title', 'Book-Author', 'Publisher'])
merging_book_ratings.head() [['ISBN', 'Book-Title', 'Average_Rating']]

#c'é da fare un ulteriore groupby

,ISBN,Book-Title,Average_Rating
3947,1565920465,!%@ (A Nutshell handbook),0.00
6818,1565920317,!%@ (A Nutshell handbook),6.00
6258,0792276833,'A Hell of a Place to Lose a Cow': An American...,0.00
6868,0792277295,'A Hell of a Place to Lose a Cow': An American...,3.40
4926,1566196256,"10,000 dreams interpreted: A dictionary of dreams",7.25


### 6. For each book in `merged books` compute the minimum and maximum of the average ratings over all corresponding books in the `books` dataset.

Hence for each book in `merged books` we will have exactly two values (a minimum and a maximum)

In [21]:
merging_book_ratings.groupby(['Book-Title', 'Book-Author', 'Publisher'], as_index=False).agg({'Average_Rating' : ['min', 'max']}).head()

Book-Title              Book-Author  \
                                                                                
0                          !%@ (A Nutshell handbook)            Donnalyn Frey   
1  'A Hell of a Place to Lose a Cow': An American...              Tim Brookes   
2  10,000 dreams interpreted: A dictionary of dreams  Gustavus Hindman Miller   
3                                   101 Famous Poems              Roy J. Cook   
4                 15 Houseplants Even You Can't Kill                Joe Elder   

                        Publisher Average_Rating        
                                             min   max  
0                        O'Reilly       0.000000  6.00  
1             National Geographic       0.000000  3.40  
2       Barnes &amp; Nobles Books       6.666667  7.25  
3  McGraw-Hill/Contemporary Books       0.000000  5.00  
4               Berkley Pub Group       0.000000  0.00

Sfruttando il dataset del punto precedente facciamo un groupby e calcoliamo max e min del valore medio con la funzione agg

### 7. For each book in `goodbooks`, compute the list of its authors. Assuming that the number of reviews with a text (column `work_text_reviews_count`) is split equally among all authors, find for each authors the total number of reviews with a text. We will call this quantity the *shared number of reviews with a text*.

Creiamo una nuova colonna chiamata authors_splitted, ottenuta con la funzione split, che contiene gli autori come lista per ogni libro.
Aggiungiamo poi la colonna shared_number_of_reviews_with_a_text calcolata dividendo work_text_reviews_count per in numero di autori di ogni libro (lunghezza della lista splitted_authors).

Costruiamo poi un nuovo dataset con una riga per ogni coppia libro-autore della lista splitted_authors. Infine facciamo un group by e una sum rispetto ad ogni attore per ottenere il shared_number_of_reviews_with_a_text di ogni scrittore.

In [22]:
goodbooks['authors_splitted'] = goodbooks['authors'].str.split(',')
goodbooks['shared_number_of_reviews_with_a_text'] = goodbooks['work_text_reviews_count'] / goodbooks['authors_splitted'].apply(len)

goodbooks_exploded = goodbooks.explode('authors_splitted')
goodbooks_exploded.groupby(['authors_splitted'], as_index=False)['shared_number_of_reviews_with_a_text'].sum().head()

,authors_splitted,shared_number_of_reviews_with_a_text
0,Alan R. Clarke,27890.500000
1,B.M. Mooyaart-Doubleday,6941.666667
2,Bernard Knox,1620.200000
3,Celâl Üster,15172.666667
4,David J. Skal,5754.333333


### 8. For each year of publication, determine the author that has the largest value of the shared number of reviews with a text.

Da spiegare bene..

Infine creo un altro dataset temporaneo chiamato authors_highest_reviews cosí da poter usare set_index per avere una vista piu compatta per gli anni con autori a pari merito.

In [23]:
temp = goodbooks_exploded.groupby(['original_publication_year', 'authors_splitted'], as_index=False).sum()
idx = temp.groupby(['original_publication_year'])['shared_number_of_reviews_with_a_text'].transform(max) == temp['shared_number_of_reviews_with_a_text']

authors_highest_reviews = temp[idx] [['original_publication_year', 'shared_number_of_reviews_with_a_text', 'authors_splitted']]
authors_highest_reviews.set_index(['original_publication_year', 'shared_number_of_reviews_with_a_text', 'authors_splitted']).head(10)

Empty DataFrame
Columns: []
Index: [(-720, 1620.2,  Bernard Knox), (-720, 1620.2,  E.V. Rieu), (-720, 1620.2,  Frédéric Mugler), (-720, 1620.2,  Robert Fagles), (-720, 1620.2, Homer), (1595, 7389.0,  Robert           Jackson), (1595, 7389.0, William Shakespeare), (1811, 3842.0,  Ros Ballaster), (1811, 3842.0,  Tony Tanner), (1811, 3842.0, Jane Austen)]

### 9. Assuming that there are no errors in the ISBN fields, find the books in both datasets, and compute the difference of average rating according to the ratings and the goodratings datasets

Scalo i voti del dataset Books perché in scala 10

In [24]:
books_goodbooks = pd.merge(books_avg, goodbooks, left_on = 'ISBN', right_on = 'isbn')

books_goodbooks['Book_Average_Rating'] = books_goodbooks['Book_Average_Rating'] / 2
books_goodbooks['Difference'] = books_goodbooks['Book_Average_Rating'] - books_goodbooks['average_rating']

books_goodbooks.rename(columns={'average_rating': 'Goodbook_Average_Rating'}, inplace=True)
books_goodbooks[['ISBN', 'Book-Title', 'Book_Average_Rating', 'Goodbook_Average_Rating', 'Difference']]

,ISBN,Book-Title,Book_Average_Rating,Goodbook_Average_Rating,Difference
0,014028009X,Bridget Jones's Diary,1.875926,3.75,-1.874074
1,043965548X,Harry Potter and the Prisoner of Azkaban (Harr...,1.766667,4.53,-2.763333
2,1400032717,The Curious Incident of the Dog in the Night-T...,2.406593,3.85,-1.443407
3,1594480001,The Kite Runner,1.200000,4.26,-3.060000
4,014038572X,The Outsiders (Now in Speak!),2.230337,4.06,-1.829663


### 10. Split the users dataset according to the age. One dataset contains the users with unknown age, one with age 0-14, one with age 15-24, one with age 25-34, and so on.

In [25]:
users.groupby(pd.cut(users['Age'], np.arange(14, 150, 10)), as_index=False).count()

,User-ID,Location,Age,City,Region,Country
0,40001,40001,40001,40001,40001,40001
1,50767,50767,50767,50767,50766,50766
2,32690,32690,32690,32690,32690,32690
3,23152,23152,23152,23152,23152,23152
4,12493,12493,12493,12493,12493,12493
5,3596,3596,3596,3596,3596,3595
6,615,615,615,615,615,615
7,83,83,83,83,83,83
8,278,278,278,278,278,278
9,20,20,20,20,20,20


In [26]:
interval = np.arange(14,95,10)
grouped_users=list(range(len(interval)))
grouped_users[0]=users[users["Age"]<=14][["User-ID","Age"]]
grouped_users[len(interval)-1]=users[users["Age"].isnull()][["User-ID","Age"]]
for i in range(1,len(interval)-1): #va da 1 a 7
    s=users[(users["Age"]>interval[i])&(users["Age"]<=interval[i+1])][["User-ID","Age"]]
    grouped_users[i]=s

grouped_users[2].head()

,User-ID,Age
53,54,44
66,67,43
69,70,44
74,75,37
84,85,41


### 11. Find the books that appear only in the goodbooks datasets.

In [27]:
goodbooks[~goodbooks['isbn'].isin(books_goodbooks['ISBN'])].head() [['isbn', 'original_title']]

,isbn,original_title
0,439023483,The Hunger Games
1,439554934,Harry Potter and the Philosopher's Stone
2,316015849,Twilight
3,61120081,To Kill a Mockingbird
4,743273567,The Great Gatsby


### 12. Assuming that each pair (author, title) identifies a book, for each book find the number of times it appears in the books dataset. Which books appear the most times?

In [28]:
books_grouped = books.groupby(['Book-Title', 'Book-Author'], as_index=False)['ISBN'].count()
books_grouped.rename(columns={'ISBN':'Count'}, inplace=True)
books_grouped.head()

,Book-Title,Book-Author,Count
0,A Light in the Storm: The Civil War Diary of ...,Karen Hesse,1
1,Always Have Popsicles,Rebecca Harvin,1
2,Apple Magic (The Collector's series),Martina Boudreau,1
3,"Ask Lily (Young Women of Faith: Lily Series, ...",Nancy N. Rue,1
4,Beyond IBM: Leadership Marketing and Finance ...,Lou Mobley,1


In [29]:
books_grouped[books_grouped['Count'] == books_grouped['Count'].max()]

,Book-Title,Book-Author,Count
114024,Little Women,Louisa May Alcott,21


### 13. Find the author with the highest average rating according to the goodbooks datasets.

In [30]:
temp = goodbooks_exploded.groupby('authors_splitted', as_index=False)['average_rating'].mean()
temp[temp['average_rating'] == temp['average_rating'].max()]

,authors_splitted,average_rating
32,Rufus Beck,4.53
